In [ ]:
from kafka import KafkaConsumer
from json import loads
import pandas as pd
import time
from IPython.display import display, clear_output

# ─────────────── CONFIG ───────────────
TOPIC         = "RandomNumber"
BOOTSTRAP     = "localhost:9092"
REFRESH_EVERY = 5      # giây
MAX_SECONDS   = 2_000
# ──────────────────────────────────────

consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=BOOTSTRAP,
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode("utf-8")),
)

records = []           # lưu toàn bộ message đã nhận

print(f"Starting to consume messages from {TOPIC} … (Ctrl+C to stop)\n")

try:
    for sec in range(0, MAX_SECONDS, REFRESH_EVERY):
        # —— thu thập message trong REFRESH_EVERY giây ——
        start = time.time()
        while time.time() - start < REFRESH_EVERY:
            for _tp, msgs in consumer.poll(timeout_ms=500).items():
                for m in msgs:
                    # build dict đúng tên cột slide
                    records.append({
                        "topic"       : m.topic,
                        "offset"      : m.offset,
                        "rand_number" : m.value["number"]
                    })

        # —— hiển thị ——        
        clear_output(wait=True)
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {sec + REFRESH_EVERY}\n")

        if not records:
            print("- no records yet -")
            continue

        df = pd.DataFrame(records)

        # Bảng 1 — raw
        display(df.tail(10))

        # Bảng 2 — thêm is_even
        df2 = df.copy()
        df2["is_even"] = df2["rand_number"] % 2 == 0
        display(df2.tail(10))

except KeyboardInterrupt:
    print("break")
finally:
    consumer.close()
    print("Live view ended…")


Showing live view refreshed every 5 seconds
Seconds passed: 245



,topic,offset,rand_number
662,RandomNumber,662,23
663,RandomNumber,663,55
664,RandomNumber,664,82
665,RandomNumber,665,5
666,RandomNumber,666,71
667,RandomNumber,667,15
668,RandomNumber,668,17
669,RandomNumber,669,24
670,RandomNumber,670,45
671,RandomNumber,671,98


,topic,offset,rand_number,is_even
662,RandomNumber,662,23,False
663,RandomNumber,663,55,False
664,RandomNumber,664,82,True
665,RandomNumber,665,5,False
666,RandomNumber,666,71,False
667,RandomNumber,667,15,False
668,RandomNumber,668,17,False
669,RandomNumber,669,24,True
670,RandomNumber,670,45,False
671,RandomNumber,671,98,True
